In [2]:
%pylab inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from linearmodels.panel import PanelOLS
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

Populating the interactive namespace from numpy and matplotlib


C:\Users\Will\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
df = pd.read_csv("./SSI ACA Data/PANEL_SSI.csv")
df = df[df['State']!="United States"]
df = df.drop("Unnamed: 0", axis=1)

In [4]:
index = pd.MultiIndex.from_product([df.State.unique(), df.Year.unique()], names=["entity", "time"])

In [5]:
df = df.set_index(index)

In [6]:
#get all possible features from the feature set we've chosen
possible_xs = list(df.columns[logical_and(df.columns.str.contains("d_pct"),~df.columns.str.contains("d_pct_Bene"))])
xs = []

In [7]:
#step forward feature selection using panel OLS (built in sklearn implementation doesnt do panel data models)
highestF=0
best_x = ""
num_items = len(possible_xs)
for i in range(0,num_items):
    for j in range(0,len(possible_xs)):
        params = xs[:]
        params.append(possible_xs[j])
        model = PanelOLS(df.d_pct_BenePerCap, df[params], entity_effects=True)
        res = model.fit(cov_type='clustered', cluster_entity=True)
        if res.f_statistic.stat >= highestF:
            highestF = res.f_statistic.stat
            best_x = possible_xs[j]
    if xs.count(best_x)==0:
        xs.append(best_x)
        possible_xs.remove(best_x)


In [8]:
model = PanelOLS(df.d_pct_BenePerCap, df[xs], entity_effects=True)
res = model.fit(cov_type='clustered', cluster_entity=True)

In [9]:
res

Dep. Variable:,d_pct_BenePerCap,R-squared:,0.0455
Estimator:,PanelOLS,R-squared (Between):,-0.0542
No. Observations:,500,R-squared (Within):,0.0455
Date:,"Tue, Sep 26 2017",R-squared (Overall):,-0.0046
Time:,22:20:37,Log-likelihood,1214.0
Cov. Estimator:,Clustered,,
,,F-statistic:,21.386
Entities:,50,P-value,0.0000
Avg Obs:,10.0000,Distribution:,"F(1,449)"
Min Obs:,10.0000,,
Max Obs:,10.0000,F-statistic (robust):,18.196


In [10]:
#compute chow test for range of breakpoints from 2008 through 2013 inclusive
for i in range(2008, 2014):
    model = PanelOLS(df[logical_and(df["Year"]>2006,df["Year"]<=i)].d_pct_BenePerCap, df[logical_and(df["Year"]>2006,df["Year"]<=i)][xs], entity_effects=True)
    res_1 = model.fit(cov_type='clustered', cluster_entity=True)
    model = PanelOLS(df[logical_and(df["Year"]>i,df["Year"]<=2015)].d_pct_BenePerCap, df[logical_and(df["Year"]>i,df["Year"]<=2015)][xs], entity_effects=True)
    res_2 = model.fit(cov_type='clustered', cluster_entity=True)
    chow = ((res.resid_ss-(res_1.resid_ss + res_2.resid_ss))/ 2)/((res_1.resid_ss + res_2.resid_ss)/(res_1.nobs+res_2.nobs - 2*52))
    print(str(i)+" "+str(chow) + " " + str(stats.f.ppf(q=1-0.05, dfn = 2, dfd= res_1.nobs+res_2.nobs - 2*2)))

2008 53.45495544549032 3.01594468147
2009 58.22593040508771 3.01594468147
2010 67.15178231909589 3.01594468147
2011 74.10465636350376 3.01594468147
2012 100.75255322306971 3.01594468147
2013 101.84845919497775 3.01594468147


In [205]:
#unfortunately due to the minimal expanatory power of the model, the chow test detects structural breaks at each year. 
#it is highly likely this particular fixed effect model is misspecified

d_pct_EmpPerCap    450.491845
Name: parameter, dtype: float64